In [1]:
import numpy as np
import re
import networkx as nx
import random
import matplotlib.pyplot as plt
import os
import glob
from ALB_instance_tools import *
import copy

In [3]:
import yaml
NO_EQUIPMENT = 4
seed = 1
NO_WORKERS =4
NO_STATIONS = 4
WORKER_COST = 500
RECOURSE_COST = WORKER_COST * 2
TAKT_TIME = 1000
SEQUENCE_LENGTH = 4
#MODEL_MIXTURES = {'A':0.34, 'B':0.33, 'C':0.33}
# MODEL_MIXTURES = {'A':0.60, 'B':0.40}

instances = [
    {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_200.alb', 'name': 'A', 'probability': 0.6},
     #{'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_17.alb', 'name': 'C', 'probability': 0.4},
     {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_16.alb', 'name': 'B', 'probability': 0.4}]
#mm_instance = MultiModelInstance( init_type='from_yaml',mm_yaml_file='/Users/letshopethisworks2/Documents/Phd Paper material/MMABPWW/SALBP_benchmark/MM_instances/n=20_1_n=20_2_n=20_3_.yaml', sequence_length=SEQUENCE_LENGTH, max_workers=NO_WORKERS, no_stations=NO_STATIONS, worker_cost=WORKER_COST, recourse_cost=RECOURSE_COST)


In [15]:
# instance_groups = pair_instances(instances, MODEL_MIXTURES)

instance_list = read_instance_folder("SALBP_benchmark/large data set_n=100/")[26:50]
model_names = ['A', 'B']

def random_model_mixture(model_names, seed = None):
    '''Creates a dictionary where the model name are the keys and the probability is the values. The probabilities sum to 1'''
    model_mixture = {}
    random.seed(seed)
    for model_name in model_names:
        model_mixture[model_name] = random.random()
    total = sum(model_mixture.values())
    for model_name in model_mixture.keys():
        model_mixture[model_name] = model_mixture[model_name]/total
    return model_mixture

def make_instances(filepath,SALBP_instance_list,model_names,takt_time, sequence_length, max_workers, no_stations, worker_cost, recourse_cost,task_time_adjuster=change_task_times_linear, seed = None):
    for i in range(len(SALBP_instance_list)-len(model_names)+1):
        instances = SALBP_instance_list[i:i+len(model_names)]
        model_mixture = random_model_mixture(model_names, seed)
        model_dicts = make_instance_pair(instances, model_mixture)
        mm_instance = MultiModelTaskTimesInstance(model_dicts=model_dicts, takt_time=takt_time, sequence_length=sequence_length, max_workers=max_workers, no_stations=no_stations, worker_cost=worker_cost, recourse_cost=recourse_cost)
        mm_instance.genererate_task_times(change_func=task_time_adjuster)

        mm_instance.model_data_to_yaml(filepath)

fp = 'SALBP_benchmark/MM_instances/model_data/large_instances/2_models/'
#if filepath does not exist, create it
if not os.path.exists(fp):
    os.makedirs(fp)
make_instances(fp,instance_list, model_names, takt_time=TAKT_TIME, sequence_length=SEQUENCE_LENGTH, max_workers=NO_WORKERS, no_stations=NO_STATIONS, worker_cost=WORKER_COST, recourse_cost=RECOURSE_COST)

In [4]:
equip = Equipment(50,10, 4,  seed=None, mean = 200, variance = 100)
equip.to_yaml(f'SALBP_benchmark/MM_instances/equipment_data/')

doing new method
